In [ ]:
from matgen.base import CellComplex
from matgen import matutils

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Creating cell complex and accessing k-cells

In [ ]:
# Load cell complex from a `.tess` file
filename = 'examples/Voronoi_10cells/Voronoi/n10-id1.tess'
cell_complex_10 = CellComplex.from_tess_file(filename)
print(cell_complex_10)

In [ ]:
# get one or many k-cells
# >>> cell_complex.get_one(cell_type, cell_id)
# >>> cell_complex.get_many(cell_type, cell_ids)

v3 = cell_complex_10.get_one('v', 3) # 'vertex', 'v', 0
print(v3)
e2 = cell_complex_10.get_one('e', 2) # 'edge', 'e', 1
print(e2)
f5 = cell_complex_10.get_one('f', 5) # 'face', 'f', 2
print(f5)
p8 = cell_complex_10.get_one('p', 8) # 'poly', 'p', 3
print(p8)

In [ ]:
# get all k-cells

# cell_complex_10.vertices
# cell_complex_10.edges
# cell_complex_10.faces
# cell_complex_10.polyhedra

In [ ]:
# all cells "know" their own id, neighbours ids
print(f'Cell id: {v3.id},', 'neigbour ids:', v3.n_ids)

# all cells of lower order "know" their incident cells ids
# there are two options: with or without cell orientations 
print(f'Cell id: {e2.id},', 'signed incident ids:', e2.signed_incident_ids)
print(f'Cell id: {e2.id},', 'unsigned incident ids:', e2.incident_ids)

In [ ]:
# cells of higher order "know" their vertices, edges and faces ids

# f5.v_ids
# f5.e_ids

# p8.v_ids
# p8.e_ids
# p8.f_ids

In [ ]:
# cells can be drawn
cell_complex_10.plot_edges()
plt.show()

In [ ]:
# cells can be external or internal
ext_e_ids = cell_complex_10.get_external_ids('e')
int_e_ids = cell_complex_10.get_internal_ids('e')

ax = cell_complex_10.plot_edges(ext_e_ids, color='blue')
ax = cell_complex_10.plot_edges(int_e_ids, color='red', ax=ax)
plt.show()

int_f_ids = cell_complex_10.get_internal_ids('f')

cell_complex_10.plot_faces(int_f_ids, color='red')
plt.show()

# Disorientation angles

In [ ]:
filename = 'examples/pass1.tess'
cell_complex_pass1 = CellComplex.from_tess_file(filename)
print(cell_complex_pass1)

In [ ]:
# grains have crysym, oridesc and ori attributes

f2 = cell_complex_pass1.get_one('f', 2)
# f2.crysym
# f2.oridesc
# f2.ori

In [ ]:
# grain boundaries can have disorientation angle (theta) between corresponding incident grains
e42 = cell_complex_pass1.get_one('e', 42)

print(e42.incident_ids)
print(e42.theta)

In [ ]:
# theta may be loaded or calculated from grain crystal orientations 

# cell_complex = CellComplex.from_tess_file(filename, with_theta=True)
# cell_complex.set_thetas(thetas_list)
# cell_complex.set_theta_form_file(thetas_filename)

cell_complex_pass1.set_theta_from_ori()

In [ ]:
print(e42.theta) # theta in degrees

In [ ]:
# calculate disorientation angle between any two grains

f9 = cell_complex_pass1.get_one('f', 9)
f14 = cell_complex_pass1.get_one('f', 14)

matutils.dis_angle(f9, f14)

In [ ]:
# neighbors of orders higher than 1 can be calculated
# cell_complex_pass1.find_neighbors_of_order(max_order)
cell_complex_pass1.find_neighbors_of_order(8)

# they can be accessed via n{k}_ids attribute
print('Own id:', f9.n0_ids) # list with own id
print('Neighbors:', f9.n1_ids) # neighbors
print('Neighbors of neighbors:', f9.n2_ids) # again neighbors
print('Neighbors of 8 order:', f9.n8_ids) # again neighbors

In [ ]:
# All disorientation angles between grains and their neighbors of any order can be calculated
# >>> cell_complex_pass1.get_neighbor_dis_angles(order)
disangles1 = cell_complex_pass1.get_neighbor_dis_angles(1)

In [ ]:
plt.hist(disangles1, bins=32, density=True, histtype='step', label='pass1')

x = np.arange(63)
y = matutils.mackenzie(x)
plt.plot(x, y, label='random dist')
plt.legend()
plt.show()

# Cell complex characteristics

In [ ]:

print(cell_complex_pass1.get_special_ids())

# Internal grain boundaries may be set special by choosing lower threshold or upper threshold or both

# cell_complex = CellComplex.from_tess_file(filename, with_theta=True, theta_file, lower_thrd, upper_thrd)
# or
# cell_complex.set_thetas(thetas_list, lower_thrd, upper_thrd)
# or
# cell_complex.set_theta_form_file(thetas_filename, lower_thrd, upper_thrd)

# If needed, after setting theta triple junction types must be calculated
# cell_complex.set_junction_types()

In [ ]:
# Special GBs may be reset

cell_complex_pass1.reset_special(0, 15)
print(len(cell_complex_pass1.get_special_ids()))
print('p =', cell_complex_pass1.get_spec_fraction()) # Be careful with external/internal Neper issues

In [ ]:
# when special GBs set, triple junction types may be retrieved
# junction_type may be 0, 1, 2, 3
tj0_ids = cell_complex_pass1.get_junction_ids_of_type(0)
tj1_ids = cell_complex_pass1.get_junction_ids_of_type(1)
tj2_ids = cell_complex_pass1.get_junction_ids_of_type(2)
tj3_ids = cell_complex_pass1.get_junction_ids_of_type(3)
print('TJ3 =', cell_complex_pass1.get_j_fraction(3))

In [ ]:
ax = cell_complex_pass1.plot_edges(color='C0')
cell_complex_pass1.plot_edges(cell_complex_pass1.get_external_ids('e'), color='C2', ax=ax)
cell_complex_pass1.plot_edges(cell_complex_pass1.get_special_ids(), color='C1', ax=ax)
cell_complex_pass1.plot_vertices(tj3_ids, color='C1', ax=ax)
plt.show()

In [ ]:
ax = cell_complex_pass1.plot_edges(color='C0')
cell_complex_pass1.plot_edges(cell_complex_pass1.get_special_ids(), color='C1', ax=ax)
cell_complex_pass1.plot_vertices(tj3_ids, color='C1', ax=ax)
cell_complex_pass1.plot_vertices(tj1_ids, color='C2', ax=ax)
cell_complex_pass1.plot_vertices(tj0_ids, color='C0', ax=ax)
cell_complex_pass1.plot_vertices(tj2_ids, color='C3', ax=ax)
plt.xlim((0, 20))
plt.ylim((0, 20))
plt.show()

In [ ]:
cell_complex_pass1.j_tuple

In [ ]:
cell_complex_pass1.describe()

In [ ]:
cell_complex_pass1.describe(attr_list=['p', 'Sp', 'S'])